In [4]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

In [18]:
import os
import sys
import yfinance as yf
import backtrader as bt
from datetime import datetime

import pandas as pd

In [20]:
sys.path.append('/Users/wtai/Projects/Quantitative_trading/')
import src.utils as utils

In [23]:
tenb_df = utils.download_yf("TENB", "2y", "1d", "/Users/wtai/Projects/Quantitative_trading/data")

Use backtrader to backtest a simple strategy.

In [7]:
# if __name__ = '__main':
cerebro = bt.Cerebro()

print("Starting Portfolio Value: %.2f" % cerebro.broker.getvalue())
cerebro.run()
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00
